In [1]:
import os
import numpy as np
import pandas as pd
from astropy.time import Time
from astropy import units as u
from validate_findorb.test_findorb import loadOrb,testFO,getOrbHorizons

# `validate_findorb` Demo
Here we are going to demo the basic functions of `validate_findorb`. We are going to show a general test case for validating the accuracy of find_orb with the main belt asteroid *202930 Ivezic (1998 SG172)*. `validate_findorb` and its testing function `testFO()` run on a simple testing structure.

Here are the general steps,

1. Take in initial orbit at a specified t0 time.
2. Generate Ephemeris observations using find_orb from the initial orbit over the time scale term supplied.
3. Determine the orbit from the Ephemeris observations with find_orb to get the orbital state vectors at the final time of the time scale.
4. Propagate the initial orbit using find_orb to the final time.
5. Compare the orbit's state vectors from step 3 to the orbit vectors from step 4.
---
## Importing Orbit Data
First we need to import the initial orbit data (state vector and time). We can use either `loadOrb()` to load from a local csv or pandas DataFrame or `getOrbHorizons()` to query JPL Horizons for the data. Both functions give the same orbit object.



In [2]:
#Importing Locally
orbits = loadOrb('orbits1.csv')
# Ivezic is the first object in the table
ivezic = orbits[0]
# We can show it as a DataFrame with .df attribute
ivezic.df

,orbit_id,mjd_tdb,x,y,z,vx,vy,vz,H,G
0,202930 Ivezic (1998 SG172),59000.000801,0.568687,-2.49376,-0.182298,0.010407,0.003676,0.000483,16.65,0.15


In [3]:
# Querying with JPL
# You have to specify a time for the position of the asteroid
# Using astropy.Time you can easily generate a time
t0 = Time([59000], scale="utc", format="mjd")
getOrbHorizons('202930',t0).df

,orbit_id,mjd_tdb,x,y,z,vx,vy,vz,H,G
0,202930 Ivezic (1998 SG172),59000.0,0.568687,-2.49376,-0.182298,0.010407,0.003676,0.000483,16.66,0.15


Now we have our orbit object for *202930 Ivezic (1998 SG172)* named `ivezic`. Now we can start running tests on this object.

---
## Generating Testing Parameters
Here we can generate a sample of parameters to test over. `validate_findorb` can only test over different observation time terms and astrometric errors. We can define the time frames using `np.arange()` which is in terms of days and the errors in terms of milliarcseconds.

In [4]:
dts = [
# Short-term, 0-30 days with an observation every 1 day
    np.arange(0, 30, 1),
    
# Weekly, 77 days with an observation every 7 days
    np.arange(0, 77, 7),
    
# Medium-term, 365 days with an observation every 10 days
    np.arange(0, 365 + 10, 10)
    
]

# 0 mas observation error, 10 mas error, and 100 mas error
errors = [0.,10.,100.]

## Using the Tester
Now that we have our parameters we can begin using the function `testFO()`. We can define the `observatory_code` which is an MPC observatory code as "500" which is the Geocenter. One can iterate over different observatory codes by passing a list of codes to the tester.

Now we will use the first time scale (`dts[0]`) and first astrometric error (`errors[0]`) we defined above.

In [5]:
t0 = ivezic.epochs
testFO(ivezic,'500',dts[0],errors[0])

,orbit_id,observatory_code,arc_length [days],num_obs,num_obs_fit,epoch [mjd],astrometric_error [mas],delta epoch [mjd],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s],rms delta ra [arcsec],rms delta dec [arcsec],rms delta time [seconds],covariance
0,202930 Ivezic (1998 SG172),500,29.0,30,30,59029.0008,0.0,9.997748e-07,0.000046,1.589317e-08,0.000021,-0.00004,-0.000005,-8.887649e-09,1.307467e-08,1.629640e-09,5.558549e-10,1.716718e-10,0.0,"[[1.72269778462e-08, -3.19807119244e-08, -3.99..."


Thus `testFO()` returns the difference DataFrame which can be saved and analyzed over. See the documentation for a description of each column.

---

## Varying Time Terms
Here we can vary the time scales for each by giving `testFO()` the list of time scales `dts` we defined above.

In [6]:
testFO(ivezic,'500',dts,errors[0])

,orbit_id,observatory_code,arc_length [days],num_obs,num_obs_fit,epoch [mjd],astrometric_error [mas],delta epoch [mjd],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s],rms delta ra [arcsec],rms delta dec [arcsec],rms delta time [seconds],covariance
0,202930 Ivezic (1998 SG172),500,29.0,30,30,59029.0008,0.0,9.997748e-07,0.000046,1.589317e-08,0.000021,-0.000040,-5.057349e-06,-8.887649e-09,1.307467e-08,1.629640e-09,5.558549e-10,1.716718e-10,0.0,"[[1.72269778462e-08, -3.19807119244e-08, -3.99..."
1,202930 Ivezic (1998 SG172),500,70.0,11,11,59070.0008,0.0,9.997748e-07,0.000009,1.037233e-09,-0.000003,0.000008,7.432393e-07,8.109717e-10,-6.457737e-10,-3.397821e-11,1.905235e-09,6.632126e-10,0.0,"[[1.70555156196e-09, -4.71749358485e-09, -4.89..."
2,202930 Ivezic (1998 SG172),500,370.0,38,38,59370.0008,0.0,9.997748e-07,0.000280,1.257969e-08,0.000189,-0.000205,-1.919551e-05,1.233878e-08,-2.435919e-09,2.631903e-10,4.881165e-09,1.556552e-09,0.0,"[[2.55722264131e-11, 8.61391086906e-12, 1.9792..."


## Varying Over Specific Times
You can also specify specific dts for the tester to use. These must be in the format of a list of times in an `astropy.time.core.Time` object. You can make a set of specific dts for each object by passing a list of these time objects with the same lengths as the space objects being tested. Here we will test two different dts for the first two orbits in our list, *Ivezic* and *Lydia*. Thus we define our times `ob1` and `ob2`,

In [7]:
# Obtaining the first two Orbits
f2 = orbits[0:2]
# Generating the specific time object list
ob1 = t0 + [0,2,4,6,8]
ob2 = t0 + [1,3,5,7,9,11]
ob1

<Time object: scale='utc' format='mjd' value=[59000. 59002. 59004. 59006. 59008.]>

In [8]:
# Thus calling our tester
testFO(f2,'500',[ob1,ob2])

,orbit_id,observatory_code,arc_length [days],num_obs,num_obs_fit,epoch [mjd],astrometric_error [mas],delta epoch [mjd],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s],rms delta ra [arcsec],rms delta dec [arcsec],rms delta time [seconds],covariance
0,202930 Ivezic (1998 SG172),500,8.0,5,5,59008.0008,0,9.997748e-07,0.000059,5.322944e-08,0.000030,-0.000050,-0.000006,-2.746791e-08,4.528826e-08,5.278262e-09,1.650000e-10,1.612452e-11,0.0,"[[1.07771423646e-05, -1.81605398953e-05, -2.20..."
1,110 Lydia (A870 HA),500,10.0,6,6,59011.0008,0,9.997748e-07,0.000067,1.529262e-08,0.000003,-0.000067,-0.000006,-1.204443e-09,-1.511786e-08,-1.965668e-09,9.353074e-11,2.753483e-11,0.0,"[[4.07675336076e-07, -9.19824906485e-06, -7.98..."


Thus we see that the specific observation times of `ob1` were tested on *Ivezic* and the times `ob2` were tested on *Lydia*.
## Varying Time and Error Terms
We can vary the time scales and error terms by passing both lists (`dts` and `errors`) to `testFO()`,

In [7]:
testFO(ivezic,'500',dts,errors)

,orbit_id,observatory_code,arc_length [days],num_obs,num_obs_fit,epoch [mjd],astrometric_error [mas],delta epoch [mjd],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s],rms delta ra [arcsec],rms delta dec [arcsec],rms delta time [seconds],covariance
0,202930 Ivezic (1998 SG172),500,29.0,30,30,59029.0008,0.0,9.997748e-07,0.000046,1.589317e-08,0.000021,-0.000040,-5.057349e-06,-8.887649e-09,1.307467e-08,1.629640e-09,5.558549e-10,1.716718e-10,0.000000,"[[1.72269778462e-08, -3.19807119244e-08, -3.99..."
1,202930 Ivezic (1998 SG172),500,29.0,30,30,59029.0008,10.0,9.997748e-07,238.652542,1.383835e-01,-113.182460,209.112904,2.040980e+01,-6.439810e-02,1.215097e-01,1.543606e-02,8.538120e-03,9.532753e-03,5.448848,"[[1.81146580903e-11, -3.36287147248e-11, -4.20..."
2,202930 Ivezic (1998 SG172),500,29.0,30,30,59029.0008,100.0,9.997748e-07,1779.292869,7.046036e+00,-751.780479,1593.501036,2.479187e+02,3.683108e+00,-5.969077e+00,-6.719057e-01,9.276803e-02,8.850683e-02,54.346075,"[[1.82422723363e-09, -3.38655499562e-09, -4.23..."
3,202930 Ivezic (1998 SG172),500,70.0,11,11,59070.0008,0.0,9.997748e-07,0.000009,1.037233e-09,-0.000003,0.000008,7.432393e-07,8.109717e-10,-6.457737e-10,-3.397821e-11,1.905235e-09,6.632126e-10,0.000000,"[[1.70555156196e-09, -4.71749358485e-09, -4.89..."
4,202930 Ivezic (1998 SG172),500,70.0,11,11,59070.0008,10.0,9.997748e-07,72.047159,6.680264e-02,13.912987,-70.681533,-1.158805e+00,-3.806472e-02,5.376522e-02,1.108923e-02,4.156875e-03,6.652410e-03,1.980890,"[[1.10131434434e-12, -3.04617984746e-12, -3.16..."
5,202930 Ivezic (1998 SG172),500,70.0,11,11,59070.0008,100.0,9.997748e-07,1038.824017,8.093102e-02,233.332091,-1011.521367,-3.919180e+01,-7.982009e-02,-5.212415e-03,1.230506e-02,9.791949e-02,1.054458e-01,39.376621,"[[2.09550353511e-10, -5.79607195727e-10, -6.01..."
6,202930 Ivezic (1998 SG172),500,370.0,38,38,59370.0008,0.0,9.997748e-07,0.000280,1.257969e-08,0.000189,-0.000205,-1.919551e-05,1.233878e-08,-2.435919e-09,2.631903e-10,4.881165e-09,1.556552e-09,0.000000,"[[2.55722264131e-11, 8.61391086906e-12, 1.9792..."
7,202930 Ivezic (1998 SG172),500,370.0,38,38,59370.0008,10.0,9.997748e-07,5.819967,5.493235e-04,-5.617771,-1.160066,9.833161e-01,-4.881957e-04,2.370586e-04,8.499721e-05,1.124268e-02,1.053397e-02,1.877859,"[[3.45117689296e-14, 1.16227969032e-14, 2.6699..."
8,202930 Ivezic (1998 SG172),500,370.0,38,38,59370.0008,100.0,9.997748e-07,313.076648,2.470124e-02,-310.610121,-28.235314,-2.722328e+01,-2.468407e-02,-6.179475e-04,-6.827974e-04,8.240866e-02,9.379389e-02,9.213054,"[[2.61657359267e-12, 8.81381928357e-13, 2.0246..."


## Varying Observatory Codes
As mentioned, one can iterate over different observatory codes by passing a list of MPC observatory code strings. Here we use the Geocenter (500) and Gemini South Observatory (I11) codes.

In [8]:
obs = ['500','I11']
testFO(ivezic,obs,dts[0],errors[0])

,orbit_id,observatory_code,arc_length [days],num_obs,num_obs_fit,epoch [mjd],astrometric_error [mas],delta epoch [mjd],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s],rms delta ra [arcsec],rms delta dec [arcsec],rms delta time [seconds],covariance
0,202930 Ivezic (1998 SG172),500,29.0,30,30,59029.0008,0.0,9.997748e-07,0.000046,1.589317e-08,0.000021,-0.000040,-0.000005,-8.887649e-09,1.307467e-08,1.629640e-09,5.558549e-10,1.716718e-10,0.0,"[[1.72269778462e-08, -3.19807119244e-08, -3.99..."
1,202930 Ivezic (1998 SG172),I11,29.0,30,10,59029.0008,0.0,9.997748e-07,0.000614,6.318958e-08,-0.000282,0.000541,0.000066,4.635154e-08,-4.256100e-08,-5.746260e-09,4.152238e-09,1.274657e-10,0.0,"[[4.7665368019e-05, -8.84037642452e-05, -1.085..."


Wonderful! You have successfully used the tester. 

This tester can also be used for multiple asteroid orbits and the outputs will scale accordingly. For the sake of not knowing how much computing power and time the user has, I will not be demonstrating this feature here. Try running the `orbits` object above in `testFO()` for a demo.

---
## Command and Bash Script Output
You can use the `out` parameter to save all the files generated by the tester and the relevant command line calls to a local directory so the calls to find_orb can be run independently from the tester. This includes generated Bash scripts for ease of use.

---
If you have any more questions, email me aidan@b612foundation.org